# 此样例是二分类问题，建立带有一个包含四个神经元的隐藏层的神经网络
## 输入
x1, x2
## 输出
y = x1 + x2 >= 0? 1: 0;
## 实现方法
使用`torch.no_grad()`上下文管理器自动求导

In [1]:
import torch

## 产生数据
使用`torch.randn()`随机生成满足标准正态分布的张量，size为$1000\times2$。

In [2]:
x = torch.randn(1000, 2)
x

tensor([[-0.8351, -0.7431],
        [-1.4390, -1.0274],
        [-0.0455,  1.3343],
        ...,
        [ 1.1577, -0.5338],
        [ 0.2276,  0.4914],
        [-0.1262, -1.2293]])

使用`torch.sum(input, dim, keepdim=False, dtype=None)`生成label，其参数如下：
* `input`：需要求和的tensor
* `dim`：需要求和的维度
* `keepdim`：默认为False，如果为True则求和后的输出的维数与input相同

也可以使用`y[x.sum(dim=1) >= 0] = 1`生成label。

In [3]:
y = torch.zeros(1000, 1)
y[torch.sum(x, dim=1) >= 0] = 1
# y[x.sum(dim=1) >= 0] = 1
y

tensor([[0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
      

## 设置权重矩阵
为权重创建随机的tensor，设置`requires_grad=True`表示我们想要计算梯度

In [4]:
w1 = torch.randn(2, 4, requires_grad=True)
w2 = torch.randn(4, 1, requires_grad=True)

## 建立模型并训练（使用`torch.no_grad()`上下文管理器自动求导）
### 损失函数
损失函数使用`torch.nn.BCELoss()`，即Binary Cross Entrophy Loss，适用于使用Sigmoid激活函数的二分类问题。
* `input`: Tensor of arbitrary shape
* `target`: Tensor of the same shape as input

### 建立网络
隐藏层和输出层分别使用使用`torch.nn.ReLU()`和`torch.nn.Sigmoid()`激活函数。
### 自动求导
* `loss.backward()`：使用`autograd`计算反向传播。这个调用将计算`loss`对所有`requires_grad=True`的tensor的梯度。这次调用后，`w1.grad`和`w2.grad`将分别是`loss`对`w1`和`w2`的梯度张量。
* `torch_no_grad()`：使用梯度下降更新权重。对于这一步，我们只想对`w1`和`w2`的值进行原地改变；不想为更新阶段构建计算图，所以我们使用`torch.no_grad()`上下文管理器防止PyTorch为更新构建计算图

In [5]:
learning_rate = 0.1
loss_fn = torch.nn.BCELoss()

for t in range(1000):
    y_pred = torch.nn.Sigmoid()(torch.nn.ReLU()(x.mm(w1)).mm(w2))
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 2.1678683757781982
1 1.9525413513183594
2 1.7518088817596436
3 1.5650949478149414
4 1.3931316137313843
5 1.2372815608978271
6 1.0985372066497803
7 0.978456437587738
8 0.8778628706932068
9 0.7976683974266052
10 0.7356394529342651
11 0.6880718469619751
12 0.6493068933486938
13 0.6173494458198547
14 0.5897171497344971
15 0.5635875463485718
16 0.5379464030265808
17 0.5144315958023071
18 0.4929092824459076
19 0.47367027401924133
20 0.4565330445766449
21 0.44128528237342834
22 0.42754650115966797
23 0.41509759426116943
24 0.4037441313266754
25 0.3931989371776581
26 0.3834073543548584
27 0.37430495023727417
28 0.3658418655395508
29 0.3579579293727875
30 0.35058099031448364
31 0.3436724543571472
32 0.3372141718864441
33 0.3311375677585602
34 0.32538893818855286
35 0.3199581801891327
36 0.31479117274284363
37 0.30988210439682007
38 0.3052130937576294
39 0.300766259431839
40 0.29652512073516846
41 0.2924710810184479
42 0.28858140110969543
43 0.2848508059978485
44 0.2812635898590088
45 0.277814

602 0.07204841077327728
603 0.07198917120695114
604 0.0719299465417862
605 0.07187096774578094
606 0.07181205600500107
607 0.07175333052873611
608 0.07169471681118011
609 0.07163622230291367
610 0.07157789170742035
611 0.07151972502470016
612 0.07146172970533371
613 0.07140382379293442
614 0.07134611904621124
615 0.07128854840993881
616 0.0712311863899231
617 0.07117388397455215
618 0.07111675292253494
619 0.07105981558561325
620 0.07100299745798111
621 0.07094631344079971
622 0.07088977843523026
623 0.07083343714475632
624 0.07077716290950775
625 0.07072107493877411
626 0.07066516578197479
627 0.07060933858156204
628 0.07055377215147018
629 0.07049818336963654
630 0.07044293731451035
631 0.07038772851228714
632 0.07033274322748184
633 0.07027781754732132
634 0.0702231302857399
635 0.07016851007938385
636 0.07011404633522034
637 0.07005967199802399
638 0.07000546157360077
639 0.06995140016078949
640 0.06989744305610657
641 0.06984362751245499
642 0.06978993117809296
643 0.0697364136576

## 创建测试样例并进行测试

In [6]:
x_test = torch.randn(100, 2)
x_test

tensor([[-0.8703,  1.3348],
        [-1.1831, -0.1233],
        [ 1.0031,  0.3441],
        [ 2.6117,  0.6655],
        [-0.2030,  0.8302],
        [-1.2074,  1.8193],
        [-0.3917,  0.2265],
        [ 0.1800,  0.2103],
        [ 0.1941,  1.7965],
        [ 0.9008,  0.0722],
        [ 1.2939,  1.1327],
        [-0.1430, -1.2566],
        [ 0.3435, -1.8806],
        [ 0.0990, -0.0245],
        [ 0.6984,  1.2998],
        [-0.4331,  0.2477],
        [-0.3532,  1.9829],
        [-0.9675, -1.8058],
        [-1.1688,  0.0902],
        [-0.7218,  0.1545],
        [-0.2190, -0.7547],
        [-0.6000,  0.0092],
        [ 0.6871, -0.1984],
        [ 0.3517,  0.8581],
        [-1.8146, -0.3331],
        [ 0.6760, -0.8559],
        [ 0.6888,  0.5469],
        [-1.5815, -1.8339],
        [-0.5437,  1.4011],
        [ 1.0157,  1.3172],
        [ 1.2139,  0.4500],
        [ 0.2025, -0.0333],
        [-0.8618, -0.4387],
        [ 0.4600, -0.9691],
        [ 0.4368, -1.9015],
        [-0.2755, -0

In [7]:
y_test = torch.zeros(100, 1)
y_test[torch.sum(x_test, dim=1) >= 0] = 1
# y_test[x_test.sum(dim=1) >= 0] = 1
y_test

tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
      

In [8]:
y_test_pred = torch.zeros(100, 1)
y_test_pred[torch.nn.Sigmoid()(torch.nn.ReLU()(x_test.mm(w1)).mm(w2)) >= .5] = 1
y_test_pred

tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
      

In [9]:
result = torch.zeros(100, 1)
result[y_test == y_test_pred] = 1
print('accurate: {}'.format(result.sum().item() / 100))

accurate: 0.99
